In [1]:
import pandas as pd
import csv
import os
import pandas as pd
import numpy as np
import math
import os,sys
import pandas as pd
import pickle



In [3]:
test_data = pd.read_csv("/Users/kimjisun/Documents/simonKim/samba_tv/test_data.csv")
viewer_data = pd.read_csv("/Users/kimjisun/Documents/simonKim/samba_tv/viewer_data.csv")

In [7]:
print test_data.shape, viewer_data.shape

(418026, 9) (417464, 4)


In [4]:
test_data.head(3)

,viewer_id,date,tv_make,tv_size,uhd_capable,tv_provider,total_time_watched,watched,test
0,24726768,2018-01-16,Sony,70,0,Comcast,10.75,0,1
1,25001464,2018-01-18,Sony,32,0,NaN,2.75,0,0
2,28291998,2018-01-18,Sony,50,1,Dish Network,20.00,0,0


In [5]:
viewer_data.head(3)

,viewer_id,gender,age,city
0,1918165,Female,39,Dallas
1,27662619,Female,28,New York
2,5493662,Female,53,Detroit


# Test Size

We can see that it was almost 50/50 AB test, when we see the size of control and test groups

In [10]:
test_data['test'].value_counts()/418026 

0    0.51121
1    0.48879
Name: test, dtype: float64

In [17]:
test_data['test'].value_counts()

0    213699
1    204327
Name: test, dtype: int64

When we check the number of users in each group, we can see that there are no duplicates

In [22]:
test_data[['viewer_id','test']].groupby(['test'])['viewer_id'].nunique().reset_index(name='num_users')

,test,num_users
0,0,213695
1,1,204325


In [23]:
test_data[['viewer_id']]['viewer_id'].nunique()

418020

# Reproduce the negative result 

In [24]:
test_group_overall = test_data[test_data['test']==1][['viewer_id','watched']].groupby(['watched'])['viewer_id'].nunique().reset_index(name='num_users')
test_group_overall['ratio'] = test_group_overall['num_users']/204325
test_group_overall

,watched,num_users,ratio
0,0,194971,0.95422
1,1,9354,0.04578


In [25]:
control_group_overall = test_data[test_data['test']==0][['viewer_id','watched']].groupby(['watched'])['viewer_id'].nunique().reset_index(name='num_users')
control_group_overall['ratio'] = control_group_overall['num_users']/213695
control_group_overall

,watched,num_users,ratio
0,0,200248,0.937074
1,1,13447,0.062926


# Let's separate the data

## Let's merge data first.

In [26]:
merged_data = pd.merge(test_data, viewer_data, on=['viewer_id'])

In [27]:
merged_data.shape

(417470, 12)

In [28]:
merged_data[['viewer_id']]['viewer_id'].nunique()

417464

In [29]:
merged_data['test'].value_counts()

0    213143
1    204327
Name: test, dtype: int64

## By City 

We can see that almost 30% users live is New York and Los Angeles

In [40]:
by_city =merged_data[['city','viewer_id']].groupby('city')['viewer_id'].nunique().reset_index(name='num_users')
by_city['ratio']= by_city['num_users']/417464
by_city

,city,num_users,ratio
0,Atlanta,24326,0.058271
1,Boston,24418,0.058491
2,Chicago,33043,0.079152
3,Dallas,26681,0.063912
4,Detroit,17931,0.042952
5,Houston,24391,0.058427
6,Los Angeles,52513,0.125790
7,Miami,17285,0.041405
8,Minneapolis,17295,0.041429
9,New York,69893,0.167423


There are  only cotrol group users but No test group users in Los Angeles 

In [52]:
test_users_by_cities =  merged_data[merged_data['test']==1][['city','viewer_id']].groupby(['city'])['viewer_id'].nunique().reset_index(name='total_num_users_test')
control_users_by_cities =  merged_data[merged_data['test']==0][['city','viewer_id']].groupby(['city'])['viewer_id'].nunique().reset_index(name='total_num_users_control')


In [75]:
test_users_by_cities

,city,total_num_users_test
0,Atlanta,12599
1,Boston,12660
2,Chicago,16991
3,Dallas,13807
4,Detroit,9192
5,Houston,12589
6,Miami,8984
7,Minneapolis,9092
8,New York,35812
9,Philadelphia,24495


In [76]:
control_users_by_cities

,city,total_num_users_control
0,Atlanta,11727
1,Boston,11758
2,Chicago,16052
3,Dallas,12874
4,Detroit,8739
5,Houston,11802
6,Los Angeles,52513
7,Miami,8301
8,Minneapolis,8203
9,New York,34081


There are much more test group users than control group users in Seatle and Philadelphia

In [79]:
city_experiment_info = merged_data[['city','test','viewer_id']].groupby(['city','test'])['viewer_id'].nunique().reset_index(name='total_num_users_by_group')
city_experiment_info.head(5)
city_experiment_info_2 = pd.merge(city_experiment_info, by_city, on=['city'])
city_experiment_info_2['ratio'] = city_experiment_info_2['total_num_users_by_group']/city_experiment_info_2['num_users']
city_experiment_info_2

,city,test,total_num_users_by_group,num_users,ratio
0,Atlanta,0,11727,24326,0.482077
1,Atlanta,1,12599,24326,0.517923
2,Boston,0,11758,24418,0.481530
3,Boston,1,12660,24418,0.518470
4,Chicago,0,16052,33043,0.485791
5,Chicago,1,16991,33043,0.514209
6,Dallas,0,12874,26681,0.482516
7,Dallas,1,13807,26681,0.517484
8,Detroit,0,8739,17931,0.487368
9,Detroit,1,9192,17931,0.512632


In [68]:
test_by_cities = merged_data[merged_data['test']==1][['city','viewer_id','watched']].groupby(['city','watched'])['viewer_id'].nunique().reset_index(name='num_users_test')
#test_by_cities['ratio_test'] = test_by_cities['num_users_test']/204327

control_by_cities = merged_data[merged_data['test']==0][['city','viewer_id','watched']].groupby(['city','watched'])['viewer_id'].nunique().reset_index(name='num_users_control')
control_by_cities['ratio_control'] = control_by_cities['num_users_control']/213143


In [69]:
test_by_cities = pd.merge(test_by_cities, test_users_by_cities, on=['city'])
test_by_cities['ratio_test'] = test_by_cities['num_users_test']/test_by_cities['total_num_users_test']
test_by_cities.head(5)

,city,watched,num_users_test,total_num_users_test,ratio_test
0,Atlanta,0,11958,12599,0.949123
1,Atlanta,1,641,12599,0.050877
2,Boston,0,11998,12660,0.947709
3,Boston,1,662,12660,0.052291
4,Chicago,0,16113,16991,0.948326


In [74]:
test_by_cities

,city,watched,num_users_test,total_num_users_test,ratio_test
0,Atlanta,0,11958,12599,0.949123
1,Atlanta,1,641,12599,0.050877
2,Boston,0,11998,12660,0.947709
3,Boston,1,662,12660,0.052291
4,Chicago,0,16113,16991,0.948326
5,Chicago,1,878,16991,0.051674
6,Dallas,0,13147,13807,0.952198
7,Dallas,1,660,13807,0.047802
8,Detroit,0,8703,9192,0.946802
9,Detroit,1,489,9192,0.053198


In [70]:
control_by_cities = pd.merge(control_by_cities, control_users_by_cities, on=['city'])
control_by_cities['ratio_control'] = control_by_cities['num_users_control']/control_by_cities['total_num_users_control']
control_by_cities.head(5)

,city,watched,num_users_control,ratio_control,total_num_users_control
0,Atlanta,0,11138,0.949774,11727
1,Atlanta,1,589,0.050226,11727
2,Boston,0,11153,0.948546,11758
3,Boston,1,605,0.051454,11758
4,Chicago,0,15259,0.950598,16052


In [73]:
control_by_cities

,city,watched,num_users_control,ratio_control,total_num_users_control
0,Atlanta,0,11138,0.949774,11727
1,Atlanta,1,589,0.050226,11727
2,Boston,0,11153,0.948546,11758
3,Boston,1,605,0.051454,11758
4,Chicago,0,15259,0.950598,16052
5,Chicago,1,793,0.049402,16052
6,Dallas,0,12219,0.949122,12874
7,Dallas,1,655,0.050878,12874
8,Detroit,0,8329,0.953084,8739
9,Detroit,1,410,0.046916,8739


In [71]:
merged_cities = pd.merge(control_by_cities, test_by_cities, on=['city','watched'])
merged_cities['ratio_diff'] = merged_cities['ratio_test'] - merged_cities['ratio_control']

In [72]:
merged_cities[merged_cities['watched']==1]

,city,watched,num_users_control,ratio_control,total_num_users_control,num_users_test,total_num_users_test,ratio_test,ratio_diff
1,Atlanta,1,589,0.050226,11727,641,12599,0.050877,0.000651
3,Boston,1,605,0.051454,11758,662,12660,0.052291,0.000836
5,Chicago,1,793,0.049402,16052,878,16991,0.051674,0.002272
7,Dallas,1,655,0.050878,12874,660,13807,0.047802,-0.003076
9,Detroit,1,410,0.046916,8739,489,9192,0.053198,0.006282
11,Houston,1,606,0.051347,11802,662,12589,0.052586,0.001238
13,Miami,1,412,0.049633,8301,469,8984,0.052204,0.002571
15,Minneapolis,1,436,0.053151,8203,477,9092,0.052464,-0.000688
17,New York,1,1808,0.053050,34081,1791,35812,0.050011,-0.003039
19,Philadelphia,1,100,0.023469,4261,575,24495,0.023474,0.000006


In [47]:
merged_cities['ratio_diff'].describe()

count    28.000000
mean      0.008799
std       0.020974
min       0.000157
25%       0.000371
50%       0.002931
75%       0.006299
max       0.097545
Name: ratio_diff, dtype: float64

# By Gender

We can see genders are almost equally distributed. in both groups

In [80]:
by_gender =merged_data[['gender','viewer_id']].groupby('gender')['viewer_id'].nunique().reset_index(name='num_users')
by_gender


,gender,num_users
0,Female,209416
1,Male,208048


In [81]:
gender_experiment_info = merged_data[['gender','test','viewer_id']].groupby(['gender','test'])['viewer_id'].nunique().reset_index(name='total_num_users_by_group')
gender_experiment_info_2 = pd.merge(gender_experiment_info, by_gender, on=['gender'])
gender_experiment_info_2['ratio'] = gender_experiment_info_2['total_num_users_by_group']/gender_experiment_info_2['num_users']
gender_experiment_info_2

,gender,test,total_num_users_by_group,num_users,ratio
0,Female,0,107100,209416,0.511422
1,Female,1,102316,209416,0.488578
2,Male,0,106039,208048,0.509685
3,Male,1,102009,208048,0.490315


# By Age

We can see that age groups are almost equally distributed in both groups

In [87]:
merged_data['age']= (merged_data['age']).apply(lambda x: int(x/10)*10)

In [88]:
by_age =merged_data[['age','viewer_id']].groupby('age')['viewer_id'].nunique().reset_index(name='num_users')
by_age



,age,num_users
0,10,5426
1,20,80983
2,30,125263
3,40,110622
4,50,64701
5,60,22539
6,70,7930


In [90]:
age_experiment_info = merged_data[['age','test','viewer_id']].groupby(['age','test'])['viewer_id'].nunique().reset_index(name='total_num_users_by_group')
age_experiment_info_2 = pd.merge(age_experiment_info, by_age, on=['age'])
age_experiment_info_2['ratio'] = age_experiment_info_2['total_num_users_by_group']/age_experiment_info_2['num_users']
age_experiment_info_2

,age,test,total_num_users_by_group,num_users,ratio
0,10,0,2807,5426,0.517324
1,10,1,2619,5426,0.482676
2,20,0,41329,80983,0.510342
3,20,1,39654,80983,0.489658
4,30,0,63977,125263,0.510741
5,30,1,61286,125263,0.489259
6,40,0,56402,110622,0.509862
7,40,1,54220,110622,0.490138
8,50,0,32996,64701,0.509977
9,50,1,31705,64701,0.490023


# TV Make

We can see that tv_makes are almost equally distributed in both groups

In [92]:
by_tv_make =merged_data[['tv_make','viewer_id']].groupby('tv_make')['viewer_id'].nunique().reset_index(name='num_users')
by_tv_make

,tv_make,num_users
0,LG,41706
1,Philips,41774
2,Sony,271573
3,Toshiba,62416


In [94]:
tv_make_experiment_info = merged_data[['tv_make','test','viewer_id']].groupby(['tv_make','test'])['viewer_id'].nunique().reset_index(name='total_num_users_by_group')
tv_make_experiment_info_2 = pd.merge(tv_make_experiment_info, by_tv_make, on=['tv_make'])
tv_make_experiment_info_2['ratio'] = tv_make_experiment_info_2['total_num_users_by_group']/tv_make_experiment_info_2['num_users']
tv_make_experiment_info_2

,tv_make,test,total_num_users_by_group,num_users,ratio
0,LG,0,21237,41706,0.509207
1,LG,1,20469,41706,0.490793
2,Philips,0,21376,41774,0.511706
3,Philips,1,20398,41774,0.488294
4,Sony,0,138577,271573,0.510275
5,Sony,1,132996,271573,0.489725
6,Toshiba,0,31952,62416,0.511920
7,Toshiba,1,30464,62416,0.488080


# TV_SIZE

We can see that tv_sizes are almost equally distributed in both groups

In [95]:
by_tv_size =merged_data[['tv_size','viewer_id']].groupby('tv_size')['viewer_id'].nunique().reset_index(name='num_users')
by_tv_size

,tv_size,num_users
0,32,52224
1,40,52321
2,43,52120
3,50,52004
4,55,52185
5,60,52179
6,65,52233
7,70,52203


In [96]:
tv_size_experiment_info = merged_data[['tv_size','test','viewer_id']].groupby(['tv_size','test'])['viewer_id'].nunique().reset_index(name='total_num_users_by_group')
tv_size_experiment_info_2 = pd.merge(tv_size_experiment_info, by_tv_size, on=['tv_size'])
tv_size_experiment_info_2['ratio'] = tv_size_experiment_info_2['total_num_users_by_group']/tv_size_experiment_info_2['num_users']
tv_size_experiment_info_2

,tv_size,test,total_num_users_by_group,num_users,ratio
0,32,0,26652,52224,0.510340
1,32,1,25572,52224,0.489660
2,40,0,26814,52321,0.512490
3,40,1,25507,52321,0.487510
4,43,0,26675,52120,0.511800
5,43,1,25445,52120,0.488200
6,50,0,26539,52004,0.510326
7,50,1,25465,52004,0.489674
8,55,0,26606,52185,0.509840
9,55,1,25579,52185,0.490160


# uhd_capable

We can see that uhd_capables are almost equally distributed in both groups

In [97]:
by_uhd_capable =merged_data[['uhd_capable','viewer_id']].groupby('uhd_capable')['viewer_id'].nunique().reset_index(name='num_users')
by_uhd_capable

,uhd_capable,num_users
0,0,333604
1,1,83863


In [98]:
uhd_capable_experiment_info = merged_data[['uhd_capable','test','viewer_id']].groupby(['uhd_capable','test'])['viewer_id'].nunique().reset_index(name='total_num_users_by_group')
uhd_capable_experiment_info_2 = pd.merge(uhd_capable_experiment_info, by_uhd_capable, on=['uhd_capable'])
uhd_capable_experiment_info_2['ratio'] = uhd_capable_experiment_info_2['total_num_users_by_group']/uhd_capable_experiment_info_2['num_users']
uhd_capable_experiment_info_2

,uhd_capable,test,total_num_users_by_group,num_users,ratio
0,0,0,170211,333604,0.510219
1,0,1,163393,333604,0.489781
2,1,0,42931,83863,0.511918
3,1,1,40932,83863,0.488082


# TV_Provider

We can see that tv_providers are almost equally distributed in both groups

In [99]:
by_tv_provider =merged_data[['tv_provider','viewer_id']].groupby('tv_provider')['viewer_id'].nunique().reset_index(name='num_users')
by_tv_provider

,tv_provider,num_users
0,Comcast,109641
1,Cox,72700
2,DirecTV,36701
3,Dish Network,36213
4,Time Warner Cable,109569


In [100]:
tv_provider_experiment_info = merged_data[['tv_provider','test','viewer_id']].groupby(['tv_provider','test'])['viewer_id'].nunique().reset_index(name='total_num_users_by_group')
tv_provider_experiment_info_2 = pd.merge(tv_provider_experiment_info, by_tv_provider, on=['tv_provider'])
tv_provider_experiment_info_2['ratio'] = tv_provider_experiment_info_2['total_num_users_by_group']/tv_provider_experiment_info_2['num_users']
tv_provider_experiment_info_2

,tv_provider,test,total_num_users_by_group,num_users,ratio
0,Comcast,0,55829,109641,0.509198
1,Comcast,1,53812,109641,0.490802
2,Cox,0,37058,72700,0.509739
3,Cox,1,35642,72700,0.490261
4,DirecTV,0,18771,36701,0.511457
5,DirecTV,1,17930,36701,0.488543
6,Dish Network,0,18389,36213,0.507801
7,Dish Network,1,17824,36213,0.492199
8,Time Warner Cable,0,56168,109569,0.512627
9,Time Warner Cable,1,53401,109569,0.487373


# Total Time Watched

We can see that distibutions of tv_makes in test and control group are almost same.

In [105]:
by_total_time_watched =merged_data[['total_time_watched','viewer_id']]
by_total_time_watched['total_time_watched'].describe()

count    417470.000000
mean         10.040225
std           6.180136
min           0.250000
25%           5.000000
50%           9.500000
75%          14.500000
max          23.750000
Name: total_time_watched, dtype: float64

In [104]:
test_users_by_total_time_watched =  merged_data[merged_data['test']==1][['total_time_watched','viewer_id']]
control_users_by_total_time_watched =  merged_data[merged_data['test']==0][['total_time_watched','viewer_id']]



In [106]:
test_users_by_total_time_watched['total_time_watched'].describe()

count    204327.000000
mean         10.050057
std           6.174999
min           0.250000
25%           5.000000
50%           9.500000
75%          14.500000
max          23.750000
Name: total_time_watched, dtype: float64

In [107]:
control_users_by_total_time_watched['total_time_watched'].describe()

count    213143.000000
mean         10.030800
std           6.185057
min           0.250000
25%           5.000000
50%           9.500000
75%          14.500000
max          23.750000
Name: total_time_watched, dtype: float64

So we can see that the experimental design in this experment is wrong because, no users in Los Angeles were selected as a test group and more users in Seatle and Philadelphia were selected as test group. Therefore the test result could be biased. 

When we exlcude the users in Los Angeles, Philadelphia and Seattle, we can see that the ratio of watched users in the test group is higher than the ratio of watched users in the control group

In [113]:
modified_data = merged_data[~merged_data['city'].isin(['Los Angeles','Philadelphia','Seattle'])]

In [114]:
modified_data[['viewer_id','test']].groupby(['test'])['viewer_id'].nunique().reset_index(name='num_users')

,test,num_users
0,0,153519
1,1,163980


In [115]:
modified_test_group_overall = modified_data[modified_data['test']==1][['viewer_id','watched']].groupby(['watched'])['viewer_id'].nunique().reset_index(name='num_users')
modified_test_group_overall['ratio'] = modified_test_group_overall['num_users']/163980
modified_test_group_overall

,watched,num_users,ratio
0,0,155608,0.948945
1,1,8372,0.051055


In [116]:
modified_control_group_overall = modified_data[modified_data['test']==0][['viewer_id','watched']].groupby(['watched'])['viewer_id'].nunique().reset_index(name='num_users')
modified_control_group_overall['ratio'] = modified_control_group_overall['num_users']/163980
modified_control_group_overall

,watched,num_users,ratio
0,0,145664,0.888303
1,1,7855,0.047902
